<a href="https://colab.research.google.com/github/sruthi1014/Movie-Recommendation-System/blob/master/movie_recommendationsystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:

# Install Java JDK
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download Spark
!wget -q http://apache.mirrors.pair.com/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz

# Untar the zip file
!tar -xvf spark-2.4.5-bin-hadoop2.7.tgz

In [ ]:
# Importing the MovieLens dataset
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip

--2020-07-06 06:59:56--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  15.3MB/s    in 0.3s    

2020-07-06 06:59:57 (15.3 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]



In [ ]:
# Unzip the file
!unzip ml-100k.zip

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


In [ ]:
# Install findspark
!pip install -q findspark

# Setup environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Spark Session Initialization
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
# Mention the Path of the u.data file
pathToFile ='/content/ml-100k/u.data'

# Read the data file
data1 = spark.read.format('com.databricks.spark.csv').\
                               options(header='true', \
                                       delimiter='\t',\
                               inferschema='true').\
                load(pathToFile,header=False)

In [ ]:
# print the schema 
data1.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)



In [ ]:
#Check the first 20 rows of the data
data1.show()

+---+----+---+---------+
|_c0| _c1|_c2|      _c3|
+---+----+---+---------+
|196| 242|  3|881250949|
|186| 302|  3|891717742|
| 22| 377|  1|878887116|
|244|  51|  2|880606923|
|166| 346|  1|886397596|
|298| 474|  4|884182806|
|115| 265|  2|881171488|
|253| 465|  5|891628467|
|305| 451|  3|886324817|
|  6|  86|  3|883603013|
| 62| 257|  2|879372434|
|286|1014|  5|879781125|
|200| 222|  5|876042340|
|210|  40|  3|891035994|
|224|  29|  3|888104457|
|303| 785|  3|879485318|
|122| 387|  5|879270459|
|194| 274|  2|879539794|
|291|1042|  4|874834944|
|234|1184|  2|892079237|
+---+----+---+---------+
only showing top 20 rows



In [ ]:
#Rename the columns of the data
data2 = data1.withColumnRenamed("_c0","user_id").withColumnRenamed("_c1","item_id").withColumnRenamed("_c2","rating").withColumnRenamed("_c3","timestamp")

In [ ]:
#Print the Schema 
data2.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [ ]:
# We do not require timestamp column, so dropping it
data3 = data2.drop("timestamp")

#print the schema now and check that timestamp column is dropped
data3.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [ ]:
# Randomly split the data into train and test where 80% data is in train and remaining is test
train, test = data3.randomSplit([0.8, 0.2])

In [ ]:
# Build a recommendation model using Alternating Least Squares
als = ALS(userCol="user_id", itemCol="item_id", ratingCol="rating", nonnegative=True)
model = als.fit(train)

In [ ]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = nan


In [ ]:
# We got mean square error as nan previously because we did not solve the cold start problem
#what is a cold start problem ? cold-start problem refers to when items added to the catalogue have either none or very little interactions.
# We again train the model to solve the cold start problem
model = ALS(userCol="user_id", itemCol="item_id", ratingCol="rating", nonnegative=True, coldStartStrategy="drop").fit(train)

# print the new RMSE
print("New RMSE: ", evaluator.evaluate(model.transform(test)))

New RMSE:  0.9173221582712675


In [ ]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder

# Now we try to improve the performance of the original model using cross validation and solve the cold-start problem.
# we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics

model = ALS(userCol="user_id", itemCol="item_id", ratingCol="rating", nonnegative = True, coldStartStrategy="drop")

# We use a ParamGridBuilder to construct a grid of parameters to search over.
paramGrid = ParamGridBuilder() \
    .addGrid(model.regParam, [0.05,0.1, 0.01, 0.001]) \
    .addGrid(model.rank, [5, 10, 15,20]) \
    .build()
print(paramGrid)

[{Param(parent='ALS_4dcad8878e77', name='regParam', doc='regularization parameter (>= 0).'): 0.05, Param(parent='ALS_4dcad8878e77', name='rank', doc='rank of the factorization'): 5}, {Param(parent='ALS_4dcad8878e77', name='regParam', doc='regularization parameter (>= 0).'): 0.05, Param(parent='ALS_4dcad8878e77', name='rank', doc='rank of the factorization'): 10}, {Param(parent='ALS_4dcad8878e77', name='regParam', doc='regularization parameter (>= 0).'): 0.05, Param(parent='ALS_4dcad8878e77', name='rank', doc='rank of the factorization'): 15}, {Param(parent='ALS_4dcad8878e77', name='regParam', doc='regularization parameter (>= 0).'): 0.05, Param(parent='ALS_4dcad8878e77', name='rank', doc='rank of the factorization'): 20}, {Param(parent='ALS_4dcad8878e77', name='regParam', doc='regularization parameter (>= 0).'): 0.1, Param(parent='ALS_4dcad8878e77', name='rank', doc='rank of the factorization'): 5}, {Param(parent='ALS_4dcad8878e77', name='regParam', doc='regularization parameter (>= 0)

In [ ]:


# Define a cross-validator object
# A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
crossvalidation = CrossValidator(estimator = model,
                     estimatorParamMaps = paramGrid,
                     evaluator = evaluator,
                     numFolds=10)

In [ ]:
# Run cross-validation, and choose the best set of parameters.
Best_model = crossvalidation.fit(train).bestModel

In [ ]:
print(Best_model)

ALS_4dcad8878e77


In [ ]:
# Calculate the RMSE on test data using the best set of parameters obtained after cross validation
print("RMSE value after solving cold start problem is: ", evaluator.evaluate(Best_model.transform(test)))

RMSE value after solving cold start problem is:  0.9120687788565961


In [ ]:
# Get top 10 movie recommendations for all users
movie_recommendation = Best_model.recommendForAllUsers(10)
movie_recommendation.show()
import pandas as pd
movie_recommendation = movie_recommendation.toPandas()

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|    471|[[1159, 5.761569]...|
|    463|[[1449, 4.35156],...|
|    833|[[1643, 4.5823517...|
|    496|[[253, 4.5259047]...|
|    148|[[1449, 5.324946]...|
|    540|[[1449, 5.139265]...|
|    392|[[1643, 5.4307027...|
|    243|[[1449, 4.6087837...|
|    623|[[1450, 4.601461]...|
|    737|[[1449, 4.9151225...|
|    897|[[1643, 5.1036468...|
|    858|[[1664, 4.5830126...|
|     31|[[1367, 4.716853]...|
|    516|[[1449, 5.0852923...|
|    580|[[753, 4.8632565]...|
|    251|[[1643, 5.2588015...|
|    451|[[899, 4.972542],...|
|     85|[[1643, 4.4368505...|
|    137|[[1019, 5.4358582...|
|    808|[[1643, 6.216156]...|
+-------+--------------------+
only showing top 20 rows



In [ ]:
pd.set_option('display.max_colwidth', -1)
print(movie_recommendation.iloc[0,1])

[Row(item_id=1159, rating=5.761569023132324), Row(item_id=1260, rating=5.260582447052002), Row(item_id=1088, rating=5.13514518737793), Row(item_id=1472, rating=4.907766342163086), Row(item_id=538, rating=4.712424278259277), Row(item_id=1153, rating=4.696173667907715), Row(item_id=169, rating=4.690969467163086), Row(item_id=1206, rating=4.676083564758301), Row(item_id=793, rating=4.639855861663818), Row(item_id=753, rating=4.505847930908203)]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [ ]:
users = []
recommendations = []

# For the entire data iterate 
for i in range(len(movie_recommendation)):

  # First we add user_id to the list
  users.append(movie_recommendation.iloc[i,0])
  user_recs = "" 

  # Fetching only the item ID's from the recommendations
  for item in movie_recommendation.iloc[i,1]:
    user_recs = user_recs + ", " + str(item.asDict()["item_id"])

  # Append the itemID's to a list
  recommendations.append(user_recs[2:])

#Convert to dataframe
recommendations_df = pd.DataFrame(data = zip(users, recommendations), columns=["user", "itemID"])

In [ ]:
#Check the users and the top 10 movie recommendations for the first 10 users
recommendations_df.head(10)

,user,itemID
0,471,"1159, 1260, 1088, 1472, 538, 1153, 169, 1206, 793, 753"
1,463,"1449, 1137, 1388, 887, 1639, 1167, 170, 119, 1512, 1062"
2,833,"1643, 1268, 853, 1597, 1367, 346, 1368, 160, 813, 641"
3,496,"253, 1449, 320, 1137, 1388, 1240, 1005, 652, 1589, 813"
4,148,"1449, 408, 902, 1589, 169, 119, 694, 114, 113, 1512"
5,540,"1449, 1643, 119, 1398, 1639, 1122, 114, 64, 1512, 318"
6,392,"1643, 1449, 1450, 483, 1398, 1064, 119, 318, 603, 357"
7,243,"1449, 1398, 483, 1099, 134, 488, 615, 1558, 1084, 1233"
8,623,"1450, 496, 1643, 483, 50, 1449, 611, 174, 1064, 1473"
9,737,"1449, 60, 1512, 1643, 114, 1368, 59, 850, 179, 156"


In [ ]:
#write to a text file
recommendations_df.to_csv('/content/ml-100k/recommendation.txt', sep='\t', index=False)